# Host Interface


In [11]:
def reset_mcu():
    # no generic PIN in gpizero?
    from gpiozero import LED
    from time import sleep
    with LED(21) as nrst:
        nrst.off()
        sleep(0.1)
        nrst.on()
        sleep(0.5)
        
def rsync_mcu():
    from iot_device import DeviceRegistry
    dev = 'serial:///dev/serial0'
    registry = DeviceRegistry()
    registry.register(dev)
    with registry.get_device(dev) as repl:
        repl.rsync(lambda x: print(x), dry_run=False)

def exec_mcu(cmd):
    from iot_device import DeviceRegistry
    dev = 'serial:///dev/serial0'
    registry = DeviceRegistry()
    registry.register(dev)
    with registry.get_device(dev) as repl:
        res = repl.exec(cmd)
        try:
            res = res.decode()
        except:
            pass
        return res

In [12]:
print(exec_mcu("""
import sys
print('path:', sys.path)
print('platform:', sys.platform)
"""))

path: ['', '/flash', '/flash/lib', '/spi', '/spi/lib']
platform: pyboard



In [18]:
from iot_device import RemoteError

reset_mcu()
exec_mcu("a = 5")
print(exec_mcu("print('a =', a)"))

reset_mcu()

try:
    print(exec_mcu("print('a =', a)"))
except RemoteError as e:
    print("After reset, MCU throws error:\n")
    print(e)

a = 5

After reset, MCU throws error:

Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'a' isn't defined

